In [ ]:
from io import BytesIO
import json
from PIL import Image
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    ImageMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    ImageMessageContent,
    LocationMessageContent
)
import cwa
from PIL import Image
from google.cloud import storage


with open('env.json', encoding='utf-8') as f:
    env = json.load(f)


storage_client = storage.Client.from_service_account_json('gcp_key.json')
bucket = storage_client.bucket('leofrstproject')

app = Flask(__name__)

configuration = Configuration(access_token=env.get('CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(env.get('CHANNEL_SECRET'))


@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    print(event.message.text)  #
    print(event.source.user_id) #
    print(event.timestamp)
    print(event.reply_token)

    # ask = event.message.text
    # if ask == 'hello':
    #     ans = '我很好'
    # elif ask == 'hi':
    #     ans = '您哪位 ?'
    # else:
    #     ans = '我不知道妳再說甚麼'

    ask = event.message.text
    ask_map = {'hello': '我很好', 'hi': '您哪位'}
    #ans = ask_map.get(ask, '我不知道妳再說甚麼')
    ans = ask_map.get(ask)
    if not ans:
        ans = cwa.cwa2(ask, env.get('CWA_KEY'))
        ans = cwa.tostr(ans, '\n') or '無此站'

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=ans)]
            )
        )

@handler.add(MessageEvent, message=LocationMessageContent)
def handle_message(event):
    print(event.source.user_id) #
    print(event.timestamp)
    print(event.reply_token)

    print(event.message.latitude)
    print(event.message.longitude)
    site = (event.message.latitude, event.message.longitude)
    ans = cwa.cwa2(site, env.get('CWA_KEY'))
    ans = cwa.tostr(ans, '\n') or '無此站'

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=ans)]
            )
        )

@handler.add(MessageEvent, message=ImageMessageContent)
def handle_content_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        message_content = line_bot_blob_api.get_message_content(message_id=event.message.id)
        image = Image.open(BytesIO(message_content))
        text = f'{image.height} X {image.width}'

        # upload image
        blob_name = f'{event.source.user_id}_{event.message.id}'
        blob = bucket.blob(blob_name)
        blob.upload_from_string(message_content, content_type='image/jpeg')

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    TextMessage(text=text)
                ]
            )
        )


if __name__ == "__main__":
    app.run(port=8080)

SyntaxError: invalid syntax (1040141508.py, line 19)